# Convex Optimization

In [1]:
import numpy as np
import pandas as pd
import random
import cvxpy as cp
import math

### 1. Variables & Parameters initialization

- Given that the problem has n agents and m items, we assume that the solution matrix $x$ size ($n,m$) will be treated as a vector of size $n \times m$ through the optimization process
<br><br>
- We initilize variable $x$ and parameter $a, A, b$
<br><br>
- parameter $a$ is a 3-D parameter including $m$ 2-D matrix with size ($n, n$). However, it will be treated as 2-D matrix with size ($m, n \times n$) through the optimization process
<br><br>
- We also define parameter $A$ for constraint. The matrix - vector multiplication equation $A \times x = b$ should represent the first constraint of the convex problem which forces the sum of each row of matrix $x$ to be equal to 1

In [2]:
# n : Number of agents
# m : Number of items

n = 2
m = 3

A = np.zeros(shape=(n, n*m))

beg = 0
end = m
for i in np.arange(n):
    idx = np.linspace(beg, end, m, endpoint = False)
    idx = idx.astype(int)
    A[i,idx] = 1
    beg += m
    end += m
    

print(A)

[[1. 1. 1. 0. 0. 0.]
 [0. 0. 0. 1. 1. 1.]]


In [3]:
# x : The solution variable
# a : Parameter a
# A : Parameter A
# b : Parameter b, which is equal to the vector of size n including ones.

x = cp.Variable(n*m)

a = cp.Parameter((m,n*n))
A_param = cp.Parameter((n, n*m))
b = cp.Parameter(n)

A_param.value = A
b.value = np.ones(shape=(n))
a = np.random.rand(m,n*n)
a = a.reshape(m, n*n)
print(A_param.shape)
print(x.shape)
print(b.shape)
print(a)

(2, 6)
(6,)
(2,)
[[0.71260638 0.26867874 0.0076263  0.63753091]
 [0.99418498 0.05961456 0.6264384  0.70803632]
 [0.33737061 0.90725415 0.9812598  0.86493739]]


### 2. Optimization

In [4]:
constraints = [0 <= x, A_param @ x == b]

def obj(a,x):
    out = 0
    idx = np.arange(x.shape[0]) 
    
    for i in np.arange(a.shape[0]): # loop over each item i
        # extract the indices of vector x that correspond to the item i
        # size of x_idcs must be m
        x_idcs = idx[idx % m == i]
        for j in np.arange(a.shape[1]): #loop over each element of a^{i}
            # a_row corresponds to the index ji and a_col corresponds to the index ki
            a_row, a_col = (math.floor(j / n), j % n)
            if a_row != a_col:
                out += a[i][j] * cp.minimum(x[x_idcs[a_row]], x[x_idcs[a_col]])
    return out

objective = cp.Maximize(obj(a,x))

prob = cp.Problem(objective, constraints)

result = prob.solve()

            
x = x.value.reshape(n,m)
x = np.array(x)
print(x)


#selct input such that the output from convex opt and intprog are different
#


[[9.75320355e-11 9.53437389e-11 1.00000000e+00]
 [9.75320437e-11 9.53437453e-11 1.00000000e+00]]
